In [ ]:
!conda install -yc conda-forge ipython-sql

In [ ]:
%%capture
%load_ext sql
%sql sqlite:///factbook.db

## query the database to get information directly

In [ ]:
%%sql 
SELECT *
FROM sqlite_master;

## query facts table

In [ ]:
%%sql
SELECT *
FROM facts
LIMIT 5;

Here are the descriptions for some of the columns:

- name — the name of the country.
- area— the country's total area (both land and water).
- area_land — the country's land area in square kilometers.
- area_water — the country's waterarea in square kilometers.
- population — the country's population.
- population_growth— the country's population growth as a percentage.
- birth_rate — the country's birth rate, or the number of births per year per 1,000 people.
- death_rate — the country's death rate, or the number of death per year per 1,000 people

In [ ]:
%%sql
SELECT MIN(population),
    MAX(population),
    MIN(population_growth),
    MAX(population_growth)
FROM facts;

We see a few interesting things in the summary statistics on the previous screen:

- There's a country with a population of 0
- There's a country with a population of 7256490011 (or more than 7.2 billion people)

Let's use subqueries to zoom in on just these countries without using the specific values.



In [ ]:
%%sql
SELECT *
FROM facts
WHERE population = 0;

In [ ]:
%%sql
SELECT *
FROM facts
WHERE population = 7256490011;

It seems like the table contains a row for the whole world, which explains the population of over 7.2 billion. It also seems like the table contains a row for Antarctica, which explains the population of 0. This seems to match the CIA Factbook page for Antarctica:

Recompute the summary statistics you found earlier while excluding the row for the whole world

In [ ]:
%%sql
SELECT MIN(population),
    MAX(population),
    MIN(population_growth),
    MAX(population_growth)
FROM facts
WHERE name NOT LIKE 'World';

In [ ]:
%%sql
SELECT AVG(population),
    AVG(area)
FROM facts

To finish, we'll build on the query we wrote for the previous screen to find countries that are densely populated. We'll identify countries that have the following:

- Above-average values for population.
- Below-average values for area.

In [ ]:
%%sql
SELECT *
FROM facts
WHERE population > (SELECT AVG(population)
                   FROM facts
                   WHERE name NOT LIKE 'World') 
    AND area < (SELECT AVG(area)
                   FROM facts
                   WHERE name NOT LIKE 'World');

### 1.Which country has the most people? Which country has the highest growth rate?


In [ ]:
%%sql
SELECT name, max(population) AS poulation
FROM facts
GROUP BY name
HAVING name NOT LIKE 'World'
ORDER BY poulation DESC
LIMIT 1;

In [ ]:
%%sql
SELECT name, MAX(population_growth) AS population_growth
FROM facts
GROUP BY name
ORDER BY population_growth DESC
LIMIT 1;

### 2.Which countries have the highest ratios of water to land? Which countries have more water than land?

In [ ]:
%%sql
SELECT name, area
FROM facts
WHERE area = (SELECT MAX(area)
             FROM facts);

In [ ]:
%%sql
SELECT name, area_water,area_land
FROM facts
WHERE area_water > area_land;

### 3.Which countries will add the most people to their populations next year?


In [ ]:
%%sql
SELECT name,birth_rate
FROM facts
WHERE birth_rate LIKE (SELECT MAX(birth_rate)
             FROM facts);

### 4.Which countries have a higher death rate than birth rate?


In [ ]:
%%sql
SELECT name, birth_rate, death_rate,(birth_rate-death_rate) AS different
FROM facts
WHERE death_rate > birth_rate
ORDER BY different 
LIMIT 1;

### 5.Which countries have the highest population/area ratio, and how does it compare to list we found in the previous screen?

In [ ]:
%%sql
SELECT name, 
    MAX(population/area) AS 'population/area ratio'
FROM facts
WHERE name NOT LIKE 'World'
ORDER BY 'population/area ratio' DESC
LIMIT 1;